In [3]:
import logging
import time
import random
from datetime import datetime

n_campi = 'CGAR'

data_e_hora_atuais = datetime.now()
data_e_hora = data_e_hora_atuais.strftime('%Y-%m-%d_%H-%M')

logger = logging.getLogger('consolidação')
logger.setLevel(logging.DEBUG)

fHandler = logging.FileHandler(f'logs/{data_e_hora_atuais}_consolidacao.log', mode='w')
formatter = logging.Formatter('%(asctime)s - %(levelname)s - %(message)s')

fHandler.setFormatter(formatter)
logger.addHandler(fHandler)

# logging.basicConfig(level=logging.DEBUG, filename='app.log', filemode='x', format='%(asctime)s - %(levelname)s - %(message)s')
logger.info(f'Iniciando consolidação campus {n_campi}')
# logger.debug('debug message')
# logger.warning('warn message')

In [4]:
import pandas as pd

logger.info(f'carregando palnilha de levantamento fisico campus {n_campi}')
levantamento = pd.read_excel('consolidacao/levantamento/2023_levantamento_CPLT.xlsx')

logger.info(f'carregando palnilha de estoque suap campus {n_campi}')
suap = pd.read_excel('consolidacao/suap/2023-09-01_16-17_CPLT_relatorio_almox.xlsx')

logger.info(f'tratamdp dados da planilha de levantameto fisico {n_campi}')

# removendo itens nulos e deixando apenas os retificados
retificacao = levantamento[levantamento['Retificação'].str.contains('Retificação').fillna(False)]


# lista de itens vencidos
# vencido = levantamento[levantamento['Material Vencido - MV'].str.contains('x').fillna(False)]
# lista de itens não identificados
# nao_identificado = levantamento[levantamento['Sem identificação - SI'].str.contains('x').fillna(False)]

# print(levantamento.info())
# print(suap.info())

# planilha contendo apenas itens válidos
levantamento_ret = levantamento.drop_duplicates(subset=['Código'], keep='last')
levantamento_ret = levantamento_ret.rename(columns={'Código': 'codigo'})

# elemento despesa do levantamento
ed_lev = levantamento_ret['codigo'].str[3:5]


# adicionando ed ao data frame geral
# levantamento_ret['ED'] = ed_lev

logger.info(f'removendo dados duplicados')
# lista de servidores
servidores = levantamento['Endereço de e-mail'].drop_duplicates(keep='last')
# reindexando uutilizando list comprehension
servidores.index = [x for x in range(0, len(servidores))]

logger.info(f'cruzando dados entre levantamento fisico e suap')

levantamento_cross = levantamento_ret.drop(columns=['Material de pouca movimentação - PM', 'Material Vencido - MV',
                                                 'Material próximo ao vencimento - PV', 'Material danificado ou avariado - MD',
                                                 'Material Obsoleto - MO', 'Sem identificação - SI', 'Valor unitário',
                                                 'Campus', 'Retificação', 'Carimbo de data/hora'])
levantamento_cross = levantamento_cross.rename(columns={'Endereço de e-mail': 'servidor', 'Nome': 'descricao_lev',
                                                        'Quantidade': 'qtd_lev'})
levantamento_cross = levantamento_cross[['codigo', 'descricao_lev', 'qtd_lev', 'servidor']]

suap_cross = suap.rename(columns={'descricao': 'descricao_suap', 'qtd': 'qtd_suap'})
suap_cross = suap_cross.drop(columns=['vlr_total', 'vlr_unt'])
# display(suap_cross)
# display(levantamento_cross)

logger.info(f'consolidando dados')

logger.info(f'exportando planilha de consolidação')
consolidacao = pd.merge(suap_cross, levantamento_cross, on='codigo', how='outer', indicator=True)
consolidacao.to_excel(f'consolidacao/{n_campi}_consolidacao.xlsx', index=False)

logger.info(f'consolidação campus {n_campi} finalizada')

In [5]:
logger.info(f'montando planilha de ajustes ')

consolidacao['diferenca'] = consolidacao['qtd_suap'] - consolidacao['qtd_lev']
display(consolidacao)

pedidos = consolidacao[consolidacao['_merge'].str.contains('both')]
pedidos = pedidos[['codigo','ed' ,'descricao_suap', 'qtd_suap', 'qtd_lev', 'diferenca']]

acao = []
for valor in pedidos['diferenca']:
    
    if valor > 0:
        baixa = "pedido no suap"
        acao.append(baixa)
    elif valor < 0:
        baixa = "saida sem pedido"
        acao.append(baixa)
    else:
        baixa = 'adequado'
        acao.append(baixa)

logger.info(f'exportando planilha de ajustes ')

pedidos['acao'] = acao
pedidos.to_excel(f'consolidacao/pedidos/{n_campi}_pedidos.xlsx', index=False)
display(pedidos)

,codigo,descricao_suap,und,qtd_suap,ed,descricao_lev,qtd_lev,servidor,_merge,diferenca
0,30.11.922,"Álcool 70. agente desinfetante, Líquido, límpi...",BOMBONA,63,11 - MATERIAL QUIMICO,"Álcool 70. agente desinfetante, Líquido, límpi...",300.0,cmps@paulista.ifpe.edu.br,both,-237.0
1,30.16.1049,"PAPEL MILIMETRADO, MATERIAL CELULOSE VEGETAL, ...",UN,5,16 - MATERIAL DE EXPEDIENTE,"PAPEL MILIMETRADO, MATERIAL CELULOSE VEGETAL, ...",5.0,cmps@paulista.ifpe.edu.br,both,0.0
2,30.16.1076,"PAPEL CARTOLINA, MATERIAL CELULOSE VEGETAL, GR...",UN,10,16 - MATERIAL DE EXPEDIENTE,"PAPEL CARTOLINA, MATERIAL CELULOSE VEGETAL, GR...",10.0,cmps@paulista.ifpe.edu.br,both,0.0
3,30.16.1077,"PAPEL CARTOLINA, MATERIAL CELULOSE VEGETAL, GR...",PCT,16,16 - MATERIAL DE EXPEDIENTE,"PAPEL CARTOLINA, MATERIAL CELULOSE VEGETAL, GR...",15.0,cmps@paulista.ifpe.edu.br,both,1.0
4,30.16.11,CANETA ESFEROGRÁFICA (AZUL) - UNIDADE.,UN,1029,16 - MATERIAL DE EXPEDIENTE,CANETA ESFEROGRÁFICA (AZUL) - UNIDADE.,1206.0,cmps@paulista.ifpe.edu.br,both,-177.0
...,...,...,...,...,...,...,...,...,...,...
311,30.42.34,MACHADO P/ CORTAR MADEIRA (C/ CABO),UN,2,42 - FERRAMENTAS,"MACHADO P/ CORTAR MADEIRA (C/ CABO) 2,5""",2.0,cmps@paulista.ifpe.edu.br,both,0.0
312,30.42.789,BROCA DE AÇO RÁPIDO 12 MM,UN,8,42 - FERRAMENTAS,BROCA DE AÇO RÁPIDO 12 MM,7.0,cmps@paulista.ifpe.edu.br,both,1.0
313,30.42.790,BROCA AÇO RÁPIDO DE 8 MM,UN,6,42 - FERRAMENTAS,BROCA AÇO RÁPIDO DE 8 MM,6.0,cmps@paulista.ifpe.edu.br,both,0.0
314,30.42.959,"ALICATE REBITADOR, MANUAL, CABO EMBORRACHADO, ...",UN,2,42 - FERRAMENTAS,"ALICATE REBITADOR, MANUAL, CABO EMBORRACHADO, ...",2.0,cmps@paulista.ifpe.edu.br,both,0.0


,codigo,ed,descricao_suap,qtd_suap,qtd_lev,diferenca,acao
0,30.11.922,11 - MATERIAL QUIMICO,"Álcool 70. agente desinfetante, Líquido, límpi...",63,300.0,-237.0,saida sem pedido
1,30.16.1049,16 - MATERIAL DE EXPEDIENTE,"PAPEL MILIMETRADO, MATERIAL CELULOSE VEGETAL, ...",5,5.0,0.0,adequado
2,30.16.1076,16 - MATERIAL DE EXPEDIENTE,"PAPEL CARTOLINA, MATERIAL CELULOSE VEGETAL, GR...",10,10.0,0.0,adequado
3,30.16.1077,16 - MATERIAL DE EXPEDIENTE,"PAPEL CARTOLINA, MATERIAL CELULOSE VEGETAL, GR...",16,15.0,1.0,pedido no suap
4,30.16.11,16 - MATERIAL DE EXPEDIENTE,CANETA ESFEROGRÁFICA (AZUL) - UNIDADE.,1029,1206.0,-177.0,saida sem pedido
...,...,...,...,...,...,...,...
311,30.42.34,42 - FERRAMENTAS,MACHADO P/ CORTAR MADEIRA (C/ CABO),2,2.0,0.0,adequado
312,30.42.789,42 - FERRAMENTAS,BROCA DE AÇO RÁPIDO 12 MM,8,7.0,1.0,pedido no suap
313,30.42.790,42 - FERRAMENTAS,BROCA AÇO RÁPIDO DE 8 MM,6,6.0,0.0,adequado
314,30.42.959,42 - FERRAMENTAS,"ALICATE REBITADOR, MANUAL, CABO EMBORRACHADO, ...",2,2.0,0.0,adequado
